In [ ]:
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")

In [ ]:
# cargar el archivo CSV expdata.csv
df_sk = pd.read_csv('dictionarysk.csv', delimiter=';')
df_null_sk= pd.read_csv('sk.csv', delimiter=';')

In [ ]:
df_sk.head()

In [ ]:
df_null_sk.head()

In [ ]:
# 1. Extraer u_estilo de ItemName (lo que está antes del primer '/')
df_null_sk['U_Estilo'] = df_null_sk['ItemName'].str.split('/').str[0]

# 2. Condición 1: que tenga al menos un "/"
tiene_slash = df_null_sk['ItemName'].str.contains('/')

# 3. Condición 2: que u_estilo tenga al menos 2 caracteres y empiece con letra o número
formato_correcto = (
    df_null_sk['U_Estilo'].str.len() >= 2
) & (
    df_null_sk['U_Estilo'].str[0].str.isalnum()
)

# 4. Condición final: válido si cumple ambas
cond_valido = tiene_slash & formato_correcto

# 5. Separar válidos e inválidos
df_valido = df_null_sk[cond_valido].copy()
df_invalido = df_null_sk[~cond_valido].copy()

# 6. Limpiar u_estilo en los inválidos
df_invalido['U_Estilo'] = np.nan


In [ ]:
df_valido['U_Descripcion'] = df_valido['ItemName'].str.split('/').str[1]

# Eliminar "Americana" seguido de número (entero o decimal), con o sin espacio
df_valido['U_Descripcion'] = df_valido['U_Descripcion'].str.replace(
    r'(?i)americana\s*\d+(?:\.\d+)?', '', regex=True
)



In [ ]:
df_valido.head()

In [ ]:
df_sk.head()

In [ ]:
df_sk.head()

In [ ]:
# Seleccionar columnas necesarias del df_sk como referencia
df_temp = df_sk[['ItemCode', 'Empresa', 'U_Genero', 'U_Suela', 'U_Descrip_Color',
                 'U_Segmentacion_SK', 'U_Division', 'U_Temporalidad']].copy()

# Renombrar columnas para evitar colisiones en el merge
df_temp.rename(columns={
    'U_Genero': 'U_Genero_sk',
    'U_Suela': 'U_Suela_sk',
    'U_Descrip_Color': 'U_Descrip_Color_sk',
    'U_Segmentacion_SK': 'U_Segmentacion_SK_sk',  # <- corregido aquí
    'U_Division': 'U_Division_sk',
    'U_Temporalidad': 'U_Temporalidad_sk'
}, inplace=True)

# Hacer merge con df_valido por ItemCode y Empresa
df_valido = df_valido.merge(df_temp, on=['ItemCode', 'Empresa'], how='left')

# Columnas originales que deseas rellenar
columnas = ['U_Genero', 'U_Suela', 'U_Descrip_Color',
            'U_Segmentacion_SK', 'U_Division', 'U_Temporalidad']

# Rellenar donde estén nulos, si existe la columna *_sk correspondiente
for col in columnas:
    col_sk = f'{col}_sk'
    if col_sk in df_valido.columns:
        df_valido[col] = df_valido[col].combine_first(df_valido[col_sk])
        df_valido.drop(columns=[col_sk], inplace=True)

df_valido.head()


In [ ]:
df_valido.head()

In [ ]:
df_invalido.head()

In [ ]:
# Seleccionar columnas necesarias de df_sk como referencia por ItemCode
df_temp = df_sk[['ItemCode', 'Empresa', 'U_Estilo', 'U_Genero', 'U_Suela', 'U_Descrip_Color',
                 'U_Segmentacion_SK', 'U_Division', 'U_Temporalidad','U_Descripcion']].copy()

# Renombrar columnas para diferenciarlas después del merge
df_temp.rename(columns={
    'U_Estilo': 'U_Estilo_sk',
    'U_Genero': 'U_Genero_sk',
    'U_Suela': 'U_Suela_sk',
    'U_Descrip_Color': 'U_Descrip_Color_sk',
    'U_Segmentacion_SK': 'U_Segmentacion_SK_sk',
    'U_Division': 'U_Division_sk',
    'U_Temporalidad': 'U_Temporalidad_sk',
    'U_Descripcion': 'U_Descripcion_sk'
}, inplace=True)

# Merge por ItemCode y Empresa
df_invalido = df_invalido.merge(df_temp, on=['ItemCode', 'Empresa'], how='left')

# Columnas a rellenar, incluyendo U_Estilo
columnas = ['U_Estilo', 'U_Genero', 'U_Suela', 'U_Descrip_Color',
            'U_Segmentacion_SK', 'U_Division', 'U_Temporalidad','U_Descripcion']

# Rellenar solo donde estén nulos
for col in columnas:
    col_sk = f'{col}_sk'
    if col_sk in df_invalido.columns:
        df_invalido[col] = df_invalido[col].combine_first(df_invalido[col_sk])
        df_invalido.drop(columns=[col_sk], inplace=True)

df_invalido.head()


In [ ]:
# Concatenar los DataFrames válidos e inválidos
df_final = pd.concat([df_valido, df_invalido], ignore_index=True)
df_final.head()